In [1]:
import pandas as pd

In [2]:
train_df=pd.read_csv('MovieData/train_data.txt', delimiter=':::')
test_df=pd.read_csv('MovieData/test_data.txt', delimiter=':::')

C:\Users\kary1\AppData\Local\Temp\ipykernel_20640\862799125.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_df=pd.read_csv('MovieData/train_data.txt', delimiter=':::')
C:\Users\kary1\AppData\Local\Temp\ipykernel_20640\862799125.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_df=pd.read_csv('MovieData/test_data.txt', delimiter=':::')


In [3]:
from sklearn.preprocessing import LabelEncoder
train_df.head()

,Index,Name,Genre,Description
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [4]:
le=LabelEncoder()
train_df['Genre_num']=le.fit_transform(train_df['Genre'])
train_df.head()

,Index,Name,Genre,Description,Genre_num
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...,8
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...,24
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...,1
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...,8
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...,8


In [5]:
train_df['Genre_num'].value_counts()

Genre_num
8     13613
7     13096
5      7447
21     5073
13     2204
24     1591
0      1315
26     1032
18      884
9       784
2       775
14      731
19      672
20      647
1       590
6       505
3       498
22      432
23      391
10      323
16      319
15      277
4       265
12      243
11      194
17      181
25      132
Name: count, dtype: int64

In [8]:
counts=train_df['Genre_num'].value_counts()

In [9]:
from sklearn.model_selection import train_test_split
train_X,test_X,train_Y,test_Y=train_test_split(train_df['Description'],train_df['Genre_num'],test_size=0.2,stratify=train_df['Genre_num'])

In [12]:
counts=train_Y.value_counts()
max_count=counts[8]
max_count

10890

In [42]:
train=pd.concat([train_X,train_Y], axis=1)
upsampled_train=train[train.iloc[:,1]==8]
for i in range(len(counts)):
    if i!=8:
        df=train[train.iloc[:,1]==i]
        df=df.sample(counts[8],replace=True)
        upsampled_train=pd.concat([upsampled_train,df],axis=0)

In [50]:
y.value_counts()

Genre_num
8     10890
14    10890
25    10890
24    10890
23    10890
22    10890
21    10890
20    10890
19    10890
18    10890
17    10890
16    10890
15    10890
13    10890
0     10890
12    10890
11    10890
10    10890
9     10890
7     10890
6     10890
5     10890
4     10890
3     10890
2     10890
1     10890
26    10890
Name: count, dtype: int64

In [43]:
upsampled_train.shape
X=upsampled_train.iloc[:,0]
y=upsampled_train.iloc[:,1]

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

In [58]:
v=TfidfVectorizer()
train_Xv=v.fit_transform(train_X)
test_Xv=v.transform(test_X)

In [57]:
train_Xv.shape

(43371, 107356)

In [59]:
smote=SMOTE()
X_upsampled,y_upsampled=smote.fit_resample(train_Xv,train_Y)
X_upsampled.shape

(294030, 107356)

In [44]:
X.shape

(294030,)

In [67]:
model=MultinomialNB()
model.fit(X_upsampled,y_upsampled)
y_pred=model.predict(test_Xv)
print(classification_report(test_Y,y_pred))

              precision    recall  f1-score   support

           0       0.34      0.46      0.39       263
           1       0.48      0.66      0.56       118
           2       0.26      0.24      0.25       155
           3       0.27      0.23      0.25       100
           4       0.01      0.02      0.01        53
           5       0.61      0.45      0.52      1490
           6       0.14      0.22      0.17       101
           7       0.73      0.65      0.69      2619
           8       0.68      0.38      0.49      2723
           9       0.21      0.24      0.22       157
          10       0.20      0.23      0.21        65
          11       0.60      0.79      0.68        39
          12       0.11      0.14      0.12        49
          13       0.48      0.76      0.59       441
          14       0.32      0.80      0.46       146
          15       0.26      0.20      0.22        55
          16       0.09      0.14      0.11        64
          17       0.26    

In [68]:
test_df.head()

,Index,Name,Description
0,1,Edgar's Lunch (1998),"L.R. Brane loves his life - his car, his apar..."
1,2,La guerra de papá (1977),"Spain, March 1964: Quico is a very naughty ch..."
2,3,Off the Beaten Track (2010),One year in the life of Albin and his family ...
3,4,Meu Amigo Hindu (2015),"His father has died, he hasn't spoken with hi..."
4,5,Er nu zhai (1955),Before he was known internationally as a mart...


In [72]:
test_df=pd.read_csv('MovieData/test_data_solution.txt',delimiter=':::')
test_df.head()

C:\Users\kary1\AppData\Local\Temp\ipykernel_20640\1518161172.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_df=pd.read_csv('MovieData/test_data_solution.txt',delimiter=':::')


,Index,Name,Genre,Description
0,1,Edgar's Lunch (1998),thriller,"L.R. Brane loves his life - his car, his apar..."
1,2,La guerra de papá (1977),comedy,"Spain, March 1964: Quico is a very naughty ch..."
2,3,Off the Beaten Track (2010),documentary,One year in the life of Albin and his family ...
3,4,Meu Amigo Hindu (2015),drama,"His father has died, he hasn't spoken with hi..."
4,5,Er nu zhai (1955),drama,Before he was known internationally as a mart...


In [73]:
test_df['Genre_num']=le.transform(test_df['Genre'])

In [74]:
test_df.head()

,Index,Name,Genre,Description,Genre_num
0,1,Edgar's Lunch (1998),thriller,"L.R. Brane loves his life - his car, his apar...",24
1,2,La guerra de papá (1977),comedy,"Spain, March 1964: Quico is a very naughty ch...",5
2,3,Off the Beaten Track (2010),documentary,One year in the life of Albin and his family ...,7
3,4,Meu Amigo Hindu (2015),drama,"His father has died, he hasn't spoken with hi...",8
4,5,Er nu zhai (1955),drama,Before he was known internationally as a mart...,8


In [75]:
text_vec=v.transform(test_df['Description'])
text_vec.shape

(54200, 107356)

In [76]:
predictions=model.predict(text_vec)
print(classification_report(test_df['Genre_num'],predictions))

              precision    recall  f1-score   support

           0       0.34      0.49      0.40      1314
           1       0.44      0.60      0.51       590
           2       0.27      0.24      0.25       775
           3       0.30      0.23      0.26       498
           4       0.05      0.07      0.06       264
           5       0.62      0.46      0.53      7446
           6       0.15      0.24      0.19       505
           7       0.74      0.66      0.70     13096
           8       0.70      0.41      0.51     13612
           9       0.20      0.24      0.22       783
          10       0.18      0.20      0.19       322
          11       0.66      0.70      0.68       193
          12       0.08      0.09      0.09       243
          13       0.45      0.72      0.56      2204
          14       0.33      0.77      0.46       731
          15       0.15      0.13      0.14       276
          16       0.10      0.15      0.12       318
          17       0.21    